Wstęp do sztucznej inteligencji. Lista 4. Zadanie 1
Karol Janic. Czerwiec 2023. **Las decyzyjny rozpoznający ręcznie pisane cyfry**. 

In [ ]:
!pip install -q tensorflow_decision_forests

In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_decision_forests as tfdf

from google.colab import drive
import os

import cv2 as cv

Połączenie się z dyskiem Google i przejście do folderu docelowego

In [ ]:
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/wsi2023')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import zbioru danych. Podział na próbki uczące i testowe

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

Normalizacja obrazów. Przeskalowanie liczb opisujących kolor z zakresu [0,255] na [0,1]

In [ ]:
train_images = train_images.reshape(-1, 28 * 28) / 255.0
test_images = test_images.reshape(-1, 28 * 28) / 255.0

Stworzenie modelu lasu:


1.   eee

In [ ]:
def create_model():
  model = tfdf.keras.RandomForestModel(num_trees=30)
  model.compile(metrics=["accuracy"])

  return model

Trenowanie modelu

In [ ]:
model = create_model()

hist = model.fit(train_images, 
          train_labels,  
          validation_data=(test_images, test_labels))  

model.save_weights('wsi4_zad1_weights')

Use /tmp/tmp6eccwujs as temporary training directory
Reading training dataset...
Training dataset read in 0:00:24.098881. Found 60000 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(10000, shape=(), dtype=int32)
Validation dataset read in 0:00:12.796000. Found 10000 examples.
Training model...
Model trained in 0:01:02.752160
Compiling model...
Model compiled.


In [ ]:
#model = create_model()
#model.load_weights('wsi4_zad1_weights')

Ocena modelu na danych testowych

In [ ]:
loss, accuracy = model.evaluate(test_images, test_labels)

print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

313/313 [==============================] - 4s 6ms/step - loss: 0.0000e+00 - accuracy: 0.9629
Test Loss: 0.0
Test Accuracy: 0.9628999829292297


Ocena modelu na podstawie własnoręcznie przygotowanych próbek

In [ ]:
DIGITS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
SAMPLES_NUMBER = 10

successes = 0
failures = {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}

for digit in DIGITS:
  for k in range(SAMPLES_NUMBER):
    img = cv.imread(f'/content/drive/MyDrive/Colab Notebooks/wsi2023/test_digits/{digit}.{k}.png', cv.IMREAD_GRAYSCALE)
    img = img.astype('float32')
    img = img.reshape(-1, 28 * 28)
    img = 255 - img
    img /= 255

    pred = model.predict(img)
    if pred.argmax(axis=1)[0] == digit:
      successes += 1
    else:
      failures[digit].append(pred.argmax(axis=1)[0])
      #failures.append(f'/content/drive/MyDrive/Colab Notebooks/wsi2023/test_digits/{digit}.{k}.png')

print("Accuracy:", successes / len(DIGITS) * SAMPLES_NUMBER)

1/1 [==============================] - 3s 3s/step
Accuracy: 50.0


In [ ]:
for digit, predicted in failures.items():
  print(f'{digit}: {(10 - len(predicted)) / 10 * 100} %   predicted = {predicted}')

0: 100.0 %   predicted = []
1: 10.0 %   predicted = [3, 3, 3, 7, 2, 0, 8, 7, 2]
2: 70.0 %   predicted = [4, 6, 8]
3: 60.0 %   predicted = [2, 2, 1, 5]
4: 70.0 %   predicted = [7, 9, 3]
5: 70.0 %   predicted = [6, 6, 3]
6: 50.0 %   predicted = [5, 2, 5, 8, 8]
7: 10.0 %   predicted = [2, 2, 2, 6, 2, 2, 2, 2, 2]
8: 50.0 %   predicted = [2, 6, 2, 5, 0]
9: 10.0 %   predicted = [3, 8, 0, 8, 0, 3, 5, 8, 3]


Dokładność na próbkach treningowych: 96%
Dokładność na własnych obrazach: 52%

Dokładność na próbkach treningowych okazała się niższa niż ta w przypadku modelu sekwencyjnej sieci neuronowej(98%).

Jednakże dokładność na własnych obrazach okazała się lepsza: 52% w porównaniu z 40%.

0: 100.0 %  wrong predicted = []

1: 10.0 %   wrongly predicted = [3, 3, 3, 7, 2, 0, 8, 7, 2]

2: 70.0 %   wrongly predicted = [4, 6, 8]

3: 60.0 %   wrongly predicted = [2, 2, 1, 5]

4: 70.0 %   wrongly predicted = [7, 9, 3]

5: 70.0 %   wrongly predicted = [6, 6, 3]

6: 50.0 %   wrongly predicted = [5, 2, 5, 8, 8]

7: 10.0 %   wrongly predicted = [2, 2, 2, 6, 2, 2, 2, 2, 2]

8: 50.0 %   wrongly predicted = [2, 6, 2, 5, 0]

9: 10.0 %   wrongly predicted = [3, 8, 0, 8, 0, 3, 5, 8, 3]